<a href="https://colab.research.google.com/github/SantiagoRodriguez8881/Metodos1_JuanPabloMartinez_SantiagoRodriguez/blob/La-ramita-del-snat/Tarea_intergacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
# TAREA #3 // INTEGRACION

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

# Ejercicio 5:

#Vamos a definir nuestra funcion que queremos integrar:

def f(x):
    return np.e**(-x**2)

# Vamos a crear una funcion que haga el metodo del trapecio: En este caso usaremos la regla del trapecio compuesta usando 5 puntos

def trapecio(f, a, b, n):
    h = (b-a)/n
    suma = 0.5*(f(a)+f(b))
    for i in range(1,n):
        suma += f(a+i*h)
    return h*suma

estimacion = trapecio(f,0,1,5)
print(estimacion)

# Ahora vamos a integrar la funcion realmente a ver cuando da:

# Vamos a usar el paquete de integracion
from scipy.integrate import quad
valor_real, error = quad(f, 0, 1)
print(valor_real)

# Ahora vamos a ver la presicion de la aproximacion con el trapecio

Presicion = abs(valor_real-estimacion)
print(Presicion)

# Cumple el requisito de que el error debe ser menor a 0.005



0.7443683397636671
0.7468241328124271
0.00245579304875998


In [47]:
# EJERCICIO 6:

# Vamos a realizar dos estimaciones, la del trapecio y la de Simpson 1/3:

def f6(x,R=0.5,a=0.01):
  return (np.sqrt(a**2 - x**2))/(R + x)

a = 0.01

# Calculemos el valor real de la integral:

Valor,error = quad(f6, -a, a)

# Usando la regla del trapecio:

Estimacion_trapecio = trapecio(f6,-a,a,40)

# Usando la regla de Simpson 1/3:

def Simpson(f, a, b):
    h = (b-a)/2
    xm = a + (h/2)
    return (h/3)*(f6(a)+4*f6(xm)+f6(b))

Estimacion_simpson = Simpson(f6,-a,a)

# Luego de esto, vamos a calcular los errores en cada una de las aproximaciones:
Error_trapecio = (abs(Valor - Estimacion_trapecio)/Valor)*100
Error_simpson = (abs(Valor - Estimacion_simpson)/Valor)*100

print(Valor)
print(Error_trapecio)
print(Error_simpson)

# Nos damos cuenta que ambas estimaciones son bastante precisas, el error de hecho es del 0.5% aproximadamente

0.00031419068757027204
0.41799334715221453
25.754376159211102


In [48]:
# EJERCICIO 7:

# Vamos a crear nuestro conjunto de soporte para poder pintar la esfera
R = 1
X,Y = np.meshgrid(np.linspace(-R,R,100),np.linspace(-R,R,100))
def circunferencia(x,y):
  return x**2 + y**2

# Tenemos que escribir Z en terminos de X y Y para cumplir con que sea una curva en R2
Z = np.sqrt(R**2 - X**2 - Y**2)

# Filtraremos los valores donde la raíz cuadrada es imaginaria (fuera de la esfera)
Z[np.isnan(Z)] = 0

# Vamos a pintar la grafica, siendo F0 la curva en sí

F0 = np.zeros_like(X)
for i in range(X.shape[0]):
  for j in range(X.shape[1]):
    F0[i,j] = circunferencia(X[i,j],Y[i,j])

#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.plot_surface(X, Y, Z, color='b')
#ax.set_xlabel('X')
#ax.set_ylabel('Y')
#ax.set_zlabel('Z')
#plt.show()


def esfera(x, y):
    return np.where(x**2 + y**2 <= R**2, np.sqrt(R**2 - x**2 - y**2), 0)

integral_matrix = np.zeros_like(X)

def promedio(X, Y):
    dx = X[1, 0] - X[0, 0]  # Suponiendo una malla uniforme
    dy = Y[0, 1] - Y[0, 0]  # Suponiendo una malla uniforme
    area = dx * dy

    total_integral = 0
    for i in range(X.shape[0] - 1):
        for j in range(X.shape[1] - 1):
            # Vértices del cuadrado pequeño
            x1, x2 = X[i, j], X[i + 1, j]
            y1, y2 = Y[i, j], Y[i, j + 1]

            # Función en los vértices
            f00 = esfera(x1, y1)
            f01 = esfera(x1, y2)
            f10 = esfera(x2, y1)
            f11 = esfera(x2, y2)

            # Promedio de la función en los vértices
            f_avg = (f00 + f01 + f10 + f11) / 4

            # Valor integral en el cuadrado pequeño
            integral_matrix[i, j] = f_avg * area
            total_integral += integral_matrix[i, j]

    return total_integral

print(promedio(X,Y))


<ipython-input-48-066452ff2db5>:10: RuntimeWarning: invalid value encountered in sqrt
  Z = np.sqrt(R**2 - X**2 - Y**2)
<ipython-input-48-066452ff2db5>:32: RuntimeWarning: invalid value encountered in sqrt
  return np.where(x**2 + y**2 <= R**2, np.sqrt(R**2 - x**2 - y**2), 0)


0.0


In [58]:
# EJERCICIO 9:

# a ))    Queremos usar la regla de Simpson 3/8, tenemos que integrar un polinomio interpolador de grado 3:

x = sym.Symbol('x')
a = sym.Symbol('a')

# Definimos b como a mas el ultimo paso "3*h"
b = a+3 * h
h = sym.Symbol('h')
hh = (b-a)/3
X = np.array([0,h,2*h,3*h])

# Vamos a hallar el polinomio interpolador de grado 3 que pase por los puntos en X

def Lagrange(x,X,i):
  L = 1
  for j in range(len(X)):
    if i != j:
      L *= (x-X[j])/(X[i]-X[j])
  return L

def Aproximacion(X,Y):
    polinomio = 0
    for i in range(len(X)):
        polinomio += Y[i]*Lagrange(x,X,i)
    return polinomio

# Vamos a definir nuestra funcion generalizada "f9" como cualquier funcion.

f9 = sym.Function('f')(x)
Y = np.array([f9.subs(x,X[i]) for i in range(len(X))])
Aprox = Aproximacion(X,Y)

# Vamos a replazar x con -a+x para integrar, de esta forma ajustamos nuestro intervalo de integracion como [a,b].
Aprox = Aprox.subs(x,-a+x)
P = Aprox
P = sym.integrate(P,(x,a,b))
P = sym.simplify(P)
print(P)

# b ))    Queremos demostrar que los puntos intermedios son (2a + b)/3 y a + (2b)/3 respectivamente

# Dado que tenemos una distribucion equiespaciada de los puntos, los dos puntos intermedios corresponden a " a + h " y a " a+ 2h ".
# Tenemos en cuenta que h es el salto, por lo cual se define como " b - a / 3", entonces:

A, B = sym.symbols('A B')
H = (B-A)/3

h1 = A + H
h2 = A + 2*H

h1,h2

# De esta forma hemos demostrado que los puntos intermedios son "(2a + b)/3" y "a + (2b)/3" respectivamente.

3*h*(f(0) + 3*f(h) + 3*f(2*h) + f(3*h))/8


(2*A/3 + B/3, A/3 + 2*B/3)

In [50]:
# EJERCICIO 10


